In [11]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
#from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [12]:
X_train = load('trainX1.npy')
X_test = load('testX1.npy')
y_train = load('trainY1.npy')
y_test = load('testY1.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [13]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

# 1-LSTM

In [14]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [15]:
model1 = Sequential()
#forward_layer =LSTM(500,input_shape=(n_timesteps,n_features),return_sequences=True)
#backward_layer =LSTM(500,activation='relu',return_sequences=True,go_backwards=True)
#model1.add(Bidirectional(LSTM(500, input_shape=(n_timesteps,n_features))))
#model1.add(Bidirectional(LSTM(500, input_shape=(n_timesteps,n_features))))
model1.add(Bidirectional(LSTM(265, return_sequences=False), input_shape=(n_timesteps,n_features)))
#model1.add(Bidirectional(LSTM(180)))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(128, activation='relu'))
#model1.add(Dropout(0.1))
#model1.add(Dense(100, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 530)               576640    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               135936    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 118)               15222     
Total params: 760,694
Trainable params: 760,694
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = model1.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128, callbacks=callbacks)

Epoch 1/200
207/207 [==============================] - 468s 2s/step - loss: 0.2195 - accuracy: 0.9395 - val_loss: 0.2183 - val_accuracy: 0.9467
Epoch 2/200
207/207 [==============================] - 454s 2s/step - loss: 0.1641 - accuracy: 0.9525 - val_loss: 0.1885 - val_accuracy: 0.9500
Epoch 3/200
207/207 [==============================] - 507s 2s/step - loss: 0.1222 - accuracy: 0.9639 - val_loss: 0.1777 - val_accuracy: 0.9541
Epoch 4/200
207/207 [==============================] - 499s 2s/step - loss: 0.1085 - accuracy: 0.9681 - val_loss: 0.1611 - val_accuracy: 0.9573
Epoch 5/200
207/207 [==============================] - 472s 2s/step - loss: 0.0917 - accuracy: 0.9727 - val_loss: 0.1603 - val_accuracy: 0.9591
Epoch 6/200
207/207 [==============================] - 470s 2s/step - loss: 0.0646 - accuracy: 0.9811 - val_loss: 0.1366 - val_accuracy: 0.9636
Epoch 7/200
207/207 [==============================] - 465s 2s/step - loss: 0.0680 - accuracy: 0.9790 - val_loss: 0.1684 - val_accuracy:

In [18]:
loss, accuracy = model1.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9683
Testing Accuracy: 0.9110
